In [34]:
import requests
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [35]:
product_info = {'Name':[], 'Origin':[]}

In [36]:
def scrape(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [37]:
def get_max_page(url):
    soup = scrape(url)

    pages = soup.find('div', class_='pagination-links__container').find_all('a', class_='pagination-item')
    max_page = pages[-1].text.strip()

    return int(max_page)

In [38]:
def process_text(tag_key, tags):
    text = tags[tag_key].text.strip()

    if(tag_key == 'Name'):
        text.replace('AUCHAN BIO\n    ', '')

    return text

In [39]:
def get_product(url):
    driver = webdriver.Firefox()

    driver.get(url)

    products = driver.find_elements('xpath','//p[@class="product-thumbnail__description"]')

    for product in products:
        print(product.text)

    driver.quit()

In [40]:
page_counter = 1
urls = ['https://www.auchan.fr/fruits-legumes/fruits/ca-n0301?page=1']

while page_counter <= get_max_page(urls[0]):
    get_product(urls[0][:-1] + str(page_counter))
    page_counter += 1
    break

retail_df = pd.DataFrame(product_info)

retail_df

CULTIVONS LE BON Clémentines Corse bio avec feuilles
Pommes Golden
Pommes Gala
Grenade
Kaki
Oranges à déguster
Oranges à jus
Oranges à déguster
CULTIVONS LE BON Clémentines Corse avec feuilles
Clémentines à déguster
Mandarines
Citrons jaunes à jus
AUCHAN BIO CULTIVONS LE BON Pomelos rose
AUCHAN BIO Oranges à déguster


,Name,Origin


### With beautifulsoup

In [176]:
def get_product(url):
    soup = scrape(url)

    articles = soup.find_all('article', role='article')

    for article in articles:
        tags = {'Name' : article.find('p', class_='product-thumbnail__description'),
                'Origin': article.find('div', class_='product-origin'),
                'Price': article.find('div', class_='product-price bolder text-dark-color'),
                'Weight/Pieces': article.find('span', class_='product-attribute')}

        print(tags['Weight/Pieces'])

        for tag_key, tag_value in tags.items():
            if tag_value:
                product_info[tag_key].append(process_text(tag_key, tags))